# A de Agro - Pipeline de Processamento e Base de Dados - Sprint 1

# Loading data



In [ ]:
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_x = open('/content/drive/MyDrive/Satelite Data/loaded_df/X.pkl', 'rb')

X = pickle.load(file_x)

file_y = open('/content/drive/MyDrive/Satelite Data/loaded_df/Y.pkl', 'rb')

Y = pickle.load(file_y)

file_x.close()
file_y.close()


In [ ]:
import tensorflow as tf

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') >= 0.8 or logs.get('val_accuracy') >= 0.8):
      print("\nAtingímos uma precisão maior ou igual a 80.0% então podemos parar o treino!")
      self.model.stop_training = True

# Transfer Learning

In [ ]:
import numpy as np

# Assuming X is your original array with a shape of (batch_size, 208, 208, 3)
batch_size = X.shape[0]  # Get the batch size from X

# Create a new channel filled with a constant value, e.g., 1, for all images in the batch
new_channel = np.ones((batch_size, 208, 208, 1))  # Note the inclusion of batch_size

# Concatenate the new channel to the original array along the last dimension
new_X = np.concatenate((X, new_channel), axis=-1)

print(new_X.shape)  # This will now correctly print (batch_size, 208, 208, 4)

(2160, 208, 208, 4)


In [ ]:
from keras import layers
from keras import Model
from keras.models import Sequential, load_model
from keras.optimizers import RMSprop
from keras.layers import UpSampling2D, Conv2D

In [ ]:
from keras.models import Model
from keras.layers import Input

### TRANSFER LEARNING

# Load the existing model
model_path = "/content/2111a6e4-5a5a-11eb-99e0-0242ac1c0002-FCNDK6-model.h5"
model = load_model(model_path)

# New input layer with a different input size
input_layer = Input(shape=(208, 208, 4))

# Dictionary to keep track of outputs of each layer, starting with the input layer
outputs = {model.layers[0].name: input_layer}

# Loop through each layer in the original model starting from index 1 (skip input layer)
for layer in model.layers[1:]:
    # Handle the case for Concatenate layers which require multiple inputs
    if 'concat' in layer.name:  # Using a simple check on name to identify concatenate layers
        # Gather all inputs for the concatenation from previous layers' outputs
        concat_inputs = [outputs[l.name] for l in layer._inbound_nodes[0].inbound_layers]
        x = layer(concat_inputs)
    else:
        # For other layers, handle single or multiple previous layers
        prev_layers = layer._inbound_nodes[0].inbound_layers
        if isinstance(prev_layers, list):
            # If the previous layer outputs to multiple next layers
            layer_inputs = [outputs[l.name] for l in prev_layers]
            x = layer(layer_inputs[0] if len(layer_inputs) == 1 else layer_inputs)
        else:
            # Single input from the previous layer's output
            x = layer(outputs[prev_layers.name])

    # Store the output of each layer in the dictionary
    outputs[layer.name] = x

# Create a new model from the new input layer and the last layer's output
new_model = Model(inputs=input_layer, outputs=x)

# Compile the new model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
new_model.summary()


Model: "model_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 208, 208, 4)]     0         
                                                                 
 conv2d (Conv2D)             multiple                  1616      
                                                                 
 batch_normalization (Batch  multiple                  64        
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     multiple                  0         
                                                                 
 zero_padding2d_1 (ZeroPadd  multiple                  0         
 ing2D)                                                          
                                                                 
 conv2d_1 (Conv2D)           multiple                  128

In [ ]:
total_params = new_model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in new_model.trainable_weights])

print(f"Total de parâmetros no modelo é de {total_params:,}.")
print(f"Total de parâmetros que podem ser treinados no modelo são {num_trainable_params:,}.")

Total de parâmetros no modelo é de 117,746.
Total de parâmetros que podem ser treinados no modelo são 117,394.


In [ ]:
#last_desired_layer = pre_trained_model.get_layer('mixed5') #Escolha a camada para "colar" o seu modelo.
last_desired_layer = model.get_layer('conv2d_5')
# Example: Fetching output tensor from the first node
last_output = last_desired_layer.get_output_at(1)
print("Output tensor dimension for node 0:", last_output.shape)


Output tensor dimension for node 0: (None, 204, 204, 32)


In [ ]:
import tensorflow as tf

# Define F1 score function
def f1_score(y_true, y_pred):
    y_pred_binary = tf.round(y_pred)  # Convert probabilities to binary values (0 or 1)

    tp = tf.reduce_sum(tf.cast(y_true * y_pred_binary, dtype=tf.float32))
    predicted_positive = tf.reduce_sum(tf.cast(y_pred_binary, dtype=tf.float32))
    actual_positive = tf.reduce_sum(tf.cast(y_true, dtype=tf.float32))

    precision = tp / (predicted_positive + tf.keras.backend.epsilon())
    recall = tp / (actual_positive + tf.keras.backend.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    return f1


In [ ]:
last_output

<KerasTensor: shape=(None, 204, 204, 32) dtype=float32 (created by layer 'conv2d_5')>

In [ ]:
x = layers.Flatten()(last_output)
# Assuming last_output is from a convolutional layer and has 4 dimensions
x = layers.Dropout(0.3)(last_output)  # Apply dropout to the convolutional output

# Continue with convolutional layers
x = layers.Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal', name="conv2d_19")(x)
x = layers.Conv2D(2, 1, padding='same', name="conv2d_20")(x)
x = UpSampling2D(size=(2, 2))(x)
x = layers.Activation('softmax', name='softmax_activation')(x)
cut_model = Model(new_model.input, x)
cut_model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
cut_model.summary()

Model: "model_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 208, 208, 4)]     0         
                                                                 
 conv2d (Conv2D)             multiple                  1616      
                                                                 
 batch_normalization (Batch  multiple                  64        
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     multiple                  0         
                                                                 
 zero_padding2d_1 (ZeroPadd  multiple                  0         
 ing2D)                                                          
                                                                 
 conv2d_1 (Conv2D)           multiple                  128

In [ ]:
total_params = cut_model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in cut_model.trainable_weights])

print(f"Total parâmetros: {total_params:,}.")
print(f"Parâmetros treináveis: {num_trainable_params:,}.")

Total de parâmetros no modelo é de 126,866.
Total de parâmetros que podem ser treinados no modelo são 126,578.


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_X, Y, test_size=0.3)

In [ ]:
results = cut_model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1155, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1249, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 620, in update_state
        metric_obj

[MODELO EM DESENVOLVIMENTO]
Não conseguimos achar uma camada no modelo prétrinado cujo output se adequasse ao requisitado pelas últimas camadas (e de saída) do modelo base desenvolvido pelo grupo.